In [1]:
push!(LOAD_PATH, "/Users/jayyao/Documents/Research/PINN/NewGridap/Module")
using PINN
using Gridap
using GridapGmsh
using Gridap.Geometry
using NLopt
using LinearAlgebra
# Geometry parameters of the mesh
L = 4.0           # Length of the normal region
h1 = 3.0          # Height of the upper region
h2 = 1.5          # Height of the lower region above source
h3 = 1.5          # Height of the lower region below source
dpml = 1.0        # Thickness of the PML
hd = 1.0          # Height of the design region

LHp=[L/2 h1]
LHn=[L/2 hd+h2+h3]

# Characteristic length (controls the resolution, smaller the finer)
λ = 1.0           # Wavelength (aribitrary unit)
resol = 10.0      # Number of points per wavelength
l0 = λ/resol      # Normal region
ld = l0/2.0       # Design region
lpml = 2*l0       # PML 


# Physical parameters 
k = 2*π/λ        # Wave number 
ω = k            # c=1
ϵ1 = 1.0         # Relative electric permittivity for material 1
ϵ2 = 12.0        # Relative electric permittivity for material 2
μ = 1.0          # Relative magnetic permeability for all materials

# PML parameters
R = 1e-4         # Tolerence for PML reflection 
σ1 = -3/4*log(R)/dpml/√ϵ1
σ2 = -3/4*log(R)/dpml/√ϵ2
σs = [σ1 σ2]

################## Generate mesh and create model
MeshGenerator(L,h1,h2,h3,hd,dpml,l0,ld,lpml)

############## Gridap Setup ##################
model = GmshDiscreteModel("geometry.msh")

order = 1
diritags = ["BottomEdge","TopEdge", "BottomNodes", "TopNodes"]
neumanntags = ["LeftEdge","RightEdge","LeftNodes","RightNodes","LeftDEdge","RightDEdge","LeftDNodes","RightDNodes"]
sourcetags = ["SourceNodes","SourceEdge"]
designboundarytags = ["LeftDEdge","RightDEdge","TBDEdge","TBDNodes","LeftDNodes","RightDNodes"]
# Test and trial finite element function space
# Scalar-valued shape functions,
# but a complex vector of free DOFs to represent the solution.
# (this automatically leads to complex sparse matrices when assembling)
order = 1
reffe = ReferenceFE(lagrangian,Float64,order)
V = TestFESpace(model,reffe,dirichlet_tags=diritags,vector_type=Vector{ComplexF64})
U = TrialFESpace(V,[0 0 0 0])

# Piece-wise constant parameter FE function space
p_reffe = ReferenceFE(lagrangian,Float64,0)
Q = TestFESpace(model,p_reffe,vector_type=Vector{Float64})
P = TrialFESpace(Q)

# Filtered parameter FE function space
pf_reffe = ReferenceFE(lagrangian,Float64,order)
Qf = TestFESpace(model,pf_reffe,vector_type=Vector{Float64})
Pf = TrialFESpace(Qf)

############### Integration domain ################
degree = 2
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)


############### Get the sub-region ################
labels = get_face_labeling(model)
dimension = num_cell_dims(model)
tags = get_face_tag(labels,dimension)
const design_tag = get_tag_from_name(labels,"Design")
cellmask_d = get_face_mask(labels,"Design",dimension)
const target_tag = get_tag_from_name(labels,"Target")
cellmask_t = get_face_mask(labels,"Target",dimension)
Ω_d = Triangulation(model,cellmask_d)
dΩ_d = Measure(Ω_d,degree)
Ω_t = Triangulation(model,cellmask_t)
dΩ_t = Measure(Ω_t,degree)
# Number of cells in design region (number of design parameters)
np = num_cells(Ω_d)

Γ = BoundaryTriangulation(model;tags=neumanntags)
dΓ = Measure(Γ,degree)

Γ_d = BoundaryTriangulation(model;tags=designboundarytags)
dΓ_d = Measure(Γ_d,degree)

Info    : Clearing all models and views...
Info    : Done clearing all models and views
Info    : Meshing 1D...
Info    : Meshing curve 1 (Line)
Info    : Meshing curve 2 (Line)
Info    : Meshing curve 3 (Line)
Info    : Meshing curve 4 (Line)
Info    : Meshing curve 5 (Line)
Info    : Meshing curve 6 (Line)
Info    : Meshing curve 7 (Line)
Info    : Meshing curve 8 (Line)
Info    : Meshing curve 9 (Line)
Info    : Meshing curve 10 (Line)
Info    : Meshing curve 11 (Line)
Info    : Meshing curve 12 (Line)
Info    : Meshing curve 13 (Line)
Info    : Meshing curve 14 (Line)
Info    : Meshing curve 15 (Line)
Info    : Meshing curve 16 (Line)
Info    : Meshing curve 17 (Line)
Info    : Meshing curve 18 (Line)
Info    : Meshing curve 19 (Line)
Info    : Meshing curve 20 (Line)
Info    : Meshing curve 21 (Line)
Info    : Meshing curve 22 (Line)
Info    : Meshing curve 23 (Line)
Info    : Meshing curve 24 (Line)
Info    : Meshing curve 25 (Line)
Info    : Meshing curve 26 (Line)
Info    : Mes

Measure()

In [2]:
############  Optimization parameters #############
# Source (particle) locations
x0 = [0 -h2-hd]
δ = [L l0]

# Target locations


flag_f = true
flag_t = false

# Filter and threshold paramters
r = ld*2.0       # Filter radius
β = 1.0          # β∈[1,∞], threshold sharpness
η = 0.5          # η∈[0,1], threshold center


Amp = 1.0
Lt = 1.0
Ht = 1.0

opt = Opt(:LD_MMA, np)
opt.lower_bounds = 0.0
opt.upper_bounds = 1.0
opt.ftol_rel = 1e-4
#opt.stopval = 0.0505*L*h1
opt.maxeval = 200
opt.min_objective = (p,grad)->g_p(p,grad;x0,δ,Amp,r,flag_f,P,Pf,Qf,β,η,flag_t,
        ϵ1,ϵ2,μ,σs,k,LHp,LHn,dpml,hd,h1,Lt,Ht,U,V,dΩ,dΓ,dΩ_t,dΓ_d,tags,design_tag)

(g_opt,p_opt,ret) = optimize(opt, ones(np)*0.5)
#(g_opt,p_opt,ret) = optimize(opt, p)
@show numevals = opt.numevals # the number of function evaluations

p_min = p_opt
pvec = p_vec(p_min,P,tags,design_tag)
pf = Filter(pvec,r,flag_f,P,Pf,Qf,dΩ,dΓ_d)
if (flag_f)
    pfh = FEFunction(Pf,pf)
else
    pfh = FEFunction(P,pf)
end
ph = (pf->Threshold(β,η,flag_t,pf))∘pfh
A_mat = MatrixA(ph,ϵ1,ϵ2,μ,σs,k,LHp,LHn,dpml,hd,U,V,dΩ)
B_vec = MatrixB(x0,δ,2*π*Amp,V,dΩ,dΓ)

u_vec = A_mat\B_vec

uh=FEFunction(U,u_vec)
writevtk(Ω,"demo",cellfields=[
  "ϵ0"=>(ϵ1+(ϵ2-ϵ1)*FEFunction(P,p_vec(p_min,P,tags,design_tag))),
  "ϵf"=>(ϵ1+(ϵ2-ϵ1)*pfh),
  "ϵt"=>(ϵ1+(ϵ2-ϵ1)*((pf->Threshold(β,η,flag_t,pf))∘pfh)),
  "Real"=>real(uh),
  "Imag"=>imag(uh),
  "|E|^2"=>abs2(uh)])
g_u(u_vec;Amp,h1,Lt,Ht,U,V,dΩ_t)/h1/L

g_value = 0.9677110607789828
g_value = 0.9618715923832492
g_value = 0.9090258188683606
g_value = 0.7223101022868722
g_value = 0.6630781339217752
g_value = 0.6548171887062672
g_value = 0.6514150788500847
g_value = 0.6369131551560464
g_value = 0.6317254775428646
g_value = 0.6323217262974496
g_value = 0.6290821457211092
g_value = 0.6266637781819989
g_value = 0.646271263253875
g_value = 0.6245748669516278
g_value = 0.6236556189859196
g_value = 0.6348928401575067
g_value = 0.6224261065710424
g_value = 0.8596144398242523
g_value = 0.6239962052942168
g_value = 0.6210852971673875
g_value = 0.6140988730596241
g_value = 1.0240789273483517
g_value = 0.6548311040265982
g_value = 0.6115255584355782
g_value = 0.6101007041268123
g_value = 0.605729910204805
g_value = 1.0860442069488043
g_value = 0.6921096088694663
g_value = 0.6054672511745801
g_value = 0.6043735960004882
g_value = 0.6037447824745807
g_value = 0.621661241771942
g_value = 0.6028837873611619
g_value = 0.6022495631385905
g_value = 0.62235

(["demo.vtu"],)

In [ ]:
using Richardson

using Random
p0 = rand(np)
δp = rand(np)
extrapolate(δp*0.1, rtol=0) do h
    @show norm(h)
    (g_p(p0+h;x0,δ,Amp,r,flag_f,P,Pf,Qf,β,η,flag_t,
        ϵ1,ϵ2,μ,σs,k,LHp,LHn,dpml,hd,h1,Lt,Ht,U,V,dΩ,dΓ,dΩ_t,dΓ_d,tags,design_tag)-
     g_p(p0;x0,δ,Amp,r,flag_f,P,Pf,Qf,β,η,flag_t,
        ϵ1,ϵ2,μ,σs,k,LHp,LHn,dpml,hd,h1,Lt,Ht,U,V,dΩ,dΓ,dΩ_t,dΓ_d,tags,design_tag)) / norm(h)
end

In [ ]:
grad = zeros(np)
g_p(p0,grad;x0,δ,Amp,r,flag_f,P,Pf,Qf,β,η,flag_t,
        ϵ1,ϵ2,μ,σs,k,LHp,LHn,dpml,hd,h1,Lt,Ht,U,V,dΩ,dΓ,dΩ_t,dΓ_d,tags,design_tag)
grad'*δp/norm(δp)


In [ ]:
pf, pf_pullback = rrule(pf_p,p0;r,flag_f,P,Pf,Qf,dΩ,dΓ_d,tags,design_tag)
u_vec, u_pullback = rrule(u_pf,pf;x0,δ,Amp,P,Pf,β,η,flag_t,ϵ1,ϵ2,μ,σs,k,LHp,LHn,dpml,hd,U,V,dΩ,dΓ)
g, g_pullback = rrule(g_u,u_vec;Amp,h1,Lt,Ht,U,V,dΩ_t)
dgdg = 1
_, dgdu = g_pullback(dgdg)
_, dgdpf = u_pullback(dgdu)
_, dgdp = pf_pullback(dgdpf)

dgdp'*δp/norm(δp)

In [4]:
g_u(u_vec;Amp,h1,Lt,Ht,U,V,dΩ_t)/h1/L

0.04979341235618561